In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [4]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [5]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 256
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/RACNN_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [6]:
import glob
import pickle
from collections import Counter
import scipy
from  scipy import signal
from scipy.fft import fft, fftfreq

In [7]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])
    #break

In [8]:
labels_bin = []
for sub in range(32):
#for sub in range(1):
  temp = labels[sub] >= 4.5
  labels_bin.append(temp)
  #print(sum(labels_bin[sub][:, type_emotion]), end=' ')

In [9]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
NSUBJECTS = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 4.5

In [26]:
labels_3 = []
for i in range(32):
  neg = labels[i] < 3.5
  pos = labels[i] >= 5.5
  temp = np.full_like(labels[i], 2, np.int8)
  temp[neg] = 0
  temp[pos] = 1
  #print(labels[i])
  #print(temp)
  labels_3.append(temp)
  #break
labels_all_3 = np.vstack(labels_3)  

In [27]:
class EmotionDataset(Dataset):
    def __init__ (self, data, labels_3, transforms, interval, shift, inds):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       self.data_samples = []
       self.labels = []
       self.transforms = transforms
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       for sub in range(len(data)): #sub   - человек
          for nvideo in range(NVIDEOS):
             if nvideo not in inds[sub]:
                continue
             
             if labels_3[sub][nvideo, 0] == 2:
               continue
             for nsec in range(0, LEN_RECORD_IN_SECONDS - interval, shift):
                    #temp = data[sub][nvideo, :,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC].copy()
                    data_sample = np.zeros((NCANALS, NTIMES_IN_SEC + 1))
                    temp = data[sub][nvideo, :32,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC]
                    for i in range(NCANALS):
                        freq, data_sample[i] = signal.welch(temp[i], fs=128.0, window='hanning', nperseg=256, noverlap=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)
                    
                    self.data_samples.append(data_sample)
                    self.labels.append(labels_3[sub][nvideo, :])
   
       
    def __len__(self):
       result =  len(self.data_samples)
       return result

    def __getitem__(self, item):
       sample = {}   
       sample = {"labels": self.labels[item],
                 "data": self.data_samples[item]
       }


       if transforms is not None:
           for t in transforms:
                sample = t(sample)
       #print(sample)         
       return sample
                 


delta (0-3 Hz), theta (4-7
Hz), low alpha (8-9.5 Hz), high alpha (10.5-12 Hz), alpha (8-
12 Hz), low beta (13-16 Hz), mid beta (17-20 Hz), high beta
(21-29 Hz), beta (13-29 Hz), and gamma (30-50 Hz).

In [12]:
# class Compose(object):
#     def __init__(self, transforms):
#         super(Compose, self).__init__()
#         self.transforms = transforms

#     def __call__(self, sample):
#         for t in self.transforms:
#             sample = t(sample)
#         return sample
import matplotlib 
from matplotlib import pyplot as plt
 
class to_pcc_matrix_per_freq(object):
    def __init__(self):
        super(to_pcc_matrix_per_freq, self).__init__()
  
    def __call__(self, sample: dict):
         freq_data = sample['data'].copy()
         freq_ranges_gc = np.array([0, 3, 4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
         koeff = 0.5
         freq_ranges_ind = ((freq_ranges_gc)/0.5).astype(int)
         list_canals = np.arange(NCANALS)
         #random.shuffle(list_canals)
         freq_data_new = freq_data#np.sqrt(np.array(freq_data).real ** 2 + np.array(freq_data).imag ** 2)
         freq_data_new = scipy.stats.zscore(freq_data_new)
         #plt.plot(freq_data_new[0])

         pcc_matrixes = []
         for s in range(10):
            ind_begin = freq_ranges_ind[s * 2]
            ind_end = freq_ranges_ind[s * 2 + 1]
            #print(ind_begin)
            ##print(ind_end)
            #print(freq_data_new.shape)
            #print("*****")
            #pcc_matrix = np.zeros((9, 9))
            pcc_matrix = np.zeros((27, 27))
            for i in range(NCANALS):
                #print(electrode_matrix[list_electrodes[i]][0])
                #print(freq_data_new)
                pcc_matrix[electrode_matrix[list_electrodes[i]][0] * 3, electrode_matrix[list_electrodes[i]][1] * 3]  = (freq_data_new[i, ind_begin : ind_end]).sum()
            #np.corrcoef(freq_data_new[:, ind_begin : ind_end])
            # for i in range(NCANALS):
            #    for j in range(i + 1):
            #       pcc_matrix[i, j] = pcc_matrix[i, j]/(np.std(freq_data_new[i, ind_begin : ind_end]) * np.std(freq_data_new[j, ind_begin : ind_end]))
            #       pcc_matrix[j, i] = pcc_matrix[i, j]
            pcc_matrixes.append(pcc_matrix)
         sample['data'] = np.array(pcc_matrixes)
         #print(pcc_matrix)
         return(sample)




# class to_pcc_matrix(object):
#     def __init__(self):
#         super(to_pcc_matrix, self).__init__()
        

#     def __call__(self, sample: dict):
#         canal_data = sample['data'].copy()
#         list_canals = np.arange(NCANALS)
#         random.shuffle(list_canals)
#         canal_data_new = [canal_data[list_canals[i]] for i in range(32)]
#         pcc_matrix = np.cov(canal_data_new)
#         for i in range(NCANALS):
#            for j in range(i + 1):
#               pcc_matrix[i, j] = pcc_matrix[i, j]/(np.std(canal_data_new[i]) * np.std(canal_data_new[j]))
#               pcc_matrix[j, i] = pcc_matrix[i, j]
#         sample['data'] = pcc_matrix
#         #print(pcc_matrix)
#         return(sample)

class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.long),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 


transforms = [to_pcc_matrix_per_freq(),
                   ToTensor()]                       

In [13]:
def get_conv_module(n_input_channels,n_output_channels):
  conv = nn.Conv2d(n_input_channels, n_output_channels, kernel_size = 3, stride=1, padding='same')
  maxpool = nn.MaxPool2d(kernel_size = 2)
  bn = nn.BatchNorm2d(n_output_channels)
  relu = nn.ReLU()
  result = torch.nn.Sequential(conv, maxpool, bn, relu)
  return(result)

In [14]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, tures, ntimes_in_sample):
      super().__init__()
      #print("1")
      n_input_channels = [10, 32, 64]
      n_output_channels = [32, 64, 128]
      self.convs = nn.ModuleList([get_conv_module(n_input_channels[i], n_output_channels[i]) for i in range(2)])
      self.conv = nn.Conv2d(n_input_channels[2], n_output_channels[2], kernel_size = 3, stride=1, padding='same')
      self.flat = nn.Flatten(1, 3)
      #self.fc = nn.Linear(512, 2)
      self.fc = nn.Linear(4608, 2)

      
   def forward(self, input):
      #print(f"input_shape = {input.shape}")
      #input = input.unsqueeze(3)
      #input = input.permute(0, 3, 1, 2)
      #input (bs, h=32, w=32, 1)

      output = input
      for i, conv in enumerate(self.convs):
            output = conv(output)
            #print(outputs[i])
      output = self.conv(output)
      output = self.flat(output)
      #output (bs, s * 256,  h=9, w=9)
      output = self.fc(output)
      return output



In [15]:
def train(model, loader, criterion, optimizer, device, type_emotion, batch = None):
    model.train()
    train_loss = []
    inputs = []
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for batch in tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True):

            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]

            #print(batch)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            #usual cross entropy
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #trg1 = trg[:, 1:].reshape(-1)
            loss = criterion(levels_pred, trg) 

            #print("after")
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            #break
    return np.mean(train_loss)#, mid_outputs

In [16]:
def evaluate(model, loader, criterion, device, type_emotion):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(tqdm(loader, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            
            loss = criterion(levels_pred, trg) 

            epoch_loss += loss.item() 
        
    return epoch_loss / s
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report, f1_score

In [17]:
def calculate_predictions(model, loader, type_emotion, show):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(tqdm(loader, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]
           

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

        if show:    
          print(accuracy_score(real, pred)) 
          print(confusion_matrix(real, pred))  
          print(classification_report(real, pred))   
    #f1 = ((f1_score(real, pred, average = 'binary', pos_label = 0))  + (f1_score(real, pred, average = 'binary', pos_label = 1)))/2
    f1 = f1_score(real, pred, average = 'macro')
    return (accuracy_score(real, pred)) , f1        
    

In [18]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv2d(10, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (flat): Flatten(start_dim=1, end_dim=3)
  (fc): Linear(in_features=4608, out_features=2, bias=True)
)

In [20]:
criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
# print(train_dataset.cnt)
# print(val_dataset.cnt)
# print(files[ind_train])
# print(files[ind_val])

In [21]:
def train_loop(type_emotion, description , fold, nepochs = 5):
        args.epochs = nepochs
        #criterion =  fnn.mse_loss
        train_loss_min = 10000
        f1_min = -10000
        #batch = next(iter(train_dataloader))
        for epoch in range(args.epochs):
            #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
            
            train_loss = train(model, train_dataloader, criterion, optimizer ,device, type_emotion)
            #if epoch % 500 == 0:
            print(train_loss)

            if (train_loss < train_loss_min):
                train_loss_min      = train_loss
                torch.save({
                                'model_state_dict': model.state_dict(),
                                'optimizer_state_dict': optimizer.state_dict(),
                              },
                              os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", "train.tgz")
                    )  

            val_loss = evaluate(model, val_dataloader, criterion, device, type_emotion)
            # #break
            print(val_loss)
            #break

            acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, False)
            print(f1, acc)
            if (f1 > f1_min):
                f1_min      =  f1
                torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"{description}_{fold}.tgz"))

In [30]:
type_emotion = 0
from sklearn.model_selection import StratifiedKFold


import pandas as pd
interval = 3
shift = 1
k = 5
inds_train = []
inds_test = []
for sub in range(32):  
    inds_train.append([])
    inds_test.append([])
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
    for ind_train, ind_test in  balanced_split:
        #print(ind_train, ind_test)
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train[-1].append(ind_train)
        inds_test[-1].append(ind_test)

inds_train = np.array(inds_train)
inds_test = np.array(inds_test)
acc_all = []
f1_all = []
for fold in range(0, 5):    
        #print(inds_train[:, fold])
        #print(inds_test[:, fold])
        #print(inds_train[:, fold].sum())
        #print(inds_test[:, fold].sum())
        train_dataset = EmotionDataset(data, labels_3, transforms, interval, shift, inds_train[:, fold])
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=True, drop_last=True)


        val_dataset = EmotionDataset(data, labels_3, transforms, interval, shift, inds_test[:, fold])
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=False, drop_last=False)
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        description = "val_psv_noleak"
        train_loop(type_emotion, description, fold, 10)

        
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"{description}_{fold}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, True)
        print(f"f1 = {f1} acc = {acc}")
        acc_all.append(acc)
        f1_all.append(f1)
        print(acc, f1)
        pd.DataFrame(f1_all).to_csv("f1_data_val_psv_noleak_noneutral.csv")
        pd.DataFrame(acc_all).to_csv("acc_data_val_psv_noleak_noneutral.csv")

training...:   0%|          | 0/157 [00:00<?, ?it/s]

0.5942391701944315


validating...:   0%|          | 0/39 [00:00<?, ?it/s]

0.5833586656341427


predicting...:   0%|          | 0/39 [00:00<?, ?it/s]

...

0.7068266707497675


predicting...:   0%|          | 0/38 [00:00<?, ?it/s]

0.6413341613621073 0.6861014812480303


predicting...:   0%|          | 0/38 [00:00<?, ?it/s]

0.6674020380292047
[[1899 1692]
 [1474 4454]]
              precision    recall  f1-score   support

           0       0.56      0.53      0.55      3591
           1       0.72      0.75      0.74      5928

    accuracy                           0.67      9519
   macro avg       0.64      0.64      0.64      9519
weighted avg       0.66      0.67      0.67      9519

f1 = 0.6415799439736787 acc = 0.6674020380292047
0.6674020380292047 0.6415799439736787


In [ ]:
import pandas as pd

f1 = f1_data_val_psv_noleak = pd.read_csv("./RESULTS/f1_data_val_psv_noleak_common.csv")
acc = acc_data_val_psv_noleak = pd.read_csv("./RESULTS/acc_data_val_psv_noleak_common.csv")

In [ ]:
print(f1['0'].values)
print(acc['0'].values)
print(f1['0'].values.mean())
print(acc['0'].values.mean())

[0.58595713 0.62718961 0.6005465  0.56731989 0.60121557]
[0.65350877 0.6779057  0.64274945 0.61108827 0.62739857]
0.5964457401567348
0.642530153508772


In [ ]:
acc_all_subs = []
f1_all_subs = []
for sub in range(32):
    acc_sub = []
    f1_sub = []
    for fold in range(0, 5):    
        # print(inds_train[sub:sub + 1, fold])
        # print(inds_test[sub : sub + 1, fold])
        # print(inds_train[sub : sub + 1, fold].sum())
        # print(inds_test[sub : sub + 1, fold].sum())
        # train_dataset = EmotionDataset((data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, interval, shift, inds_train[sub:sub+1, fold])
        # train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
        #                           pin_memory=True, shuffle=True, drop_last=True)


        val_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, interval, shift, inds_test[sub : sub + 1, fold])
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=False, drop_last=False)
        
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # model = get_model()
        # model.apply(initialize_weights)
        # criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        # #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        # optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        description = "arousal_psv_noleak"
        # train_loop(type_emotion, description, fold, 5)пео

        
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"{description}_{fold}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, False)
        print(f"f1 = {f1} acc = {acc}")
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
    acc_all_subs.append(acc_sub)   
    f1_all_subs.append(f1_sub)   
    pd.DataFrame(f1_all_subs).to_csv("f1_data_val_psv_noleak_per_subs.csv")
    pd.DataFrame(acc_all_subs).to_csv("acc_data_val_psv_noleak_per_subs.csv")

predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.36224436939270244 acc = 0.40789473684210525
0.40789473684210525 0.36224436939270244


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.47893754791304055 acc = 0.48026315789473684
0.48026315789473684 0.47893754791304055


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

...

f1 = 0.5624640184225677 acc = 0.5942982456140351
0.5942982456140351 0.5624640184225677


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.4474736459469284 acc = 0.7368421052631579
0.7368421052631579 0.4474736459469284


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.38461538461538464 acc = 0.625
0.625 0.38461538461538464


In [ ]:
print(np.mean(np.array(f1_all_subs).mean(axis = 1)))
print(np.mean(np.array(acc_all_subs).mean(axis = 1)))

0.505734941177765
0.642530153508772


#AROUSAL

In [ ]:
type_emotion = 1
from sklearn.model_selection import StratifiedKFold


import pandas as pd
interval = 3
shift = 1
k = 5
inds_train = []
inds_test = []
for sub in range(32):  
    inds_train.append([])
    inds_test.append([])
    X = np.arange(40)
    y = np.array(labels_bin[sub][:, type_emotion])
    skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
    balanced_split = skf.split(X, y)
    acc_sub = []
    f1_sub = []
    for ind_train, ind_test in  balanced_split:
        #print(ind_train, ind_test)
        #количестов меток 1 в трейне и тесте
        #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
        #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
        inds_train[-1].append(ind_train)
        inds_test[-1].append(ind_test)

inds_train = np.array(inds_train)
inds_test = np.array(inds_test)
acc_all = []
f1_all = []
for fold in range(0, 5):    
        #print(inds_train[:, fold])
        #print(inds_test[:, fold])
        #print(inds_train[:, fold].sum())
        #print(inds_test[:, fold].sum())
        train_dataset = EmotionDataset(data, labels_bin, transforms, interval, shift, inds_train[:, fold])
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=True, drop_last=True)


        val_dataset = EmotionDataset(data, labels_bin, transforms, interval, shift, inds_test[:, fold])
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=False, drop_last=False)
        
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = get_model()
        model.apply(initialize_weights)
        criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.01)
        description = "arousal_psv_noleak"
        train_loop(type_emotion, description, fold, 20)

        
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"{description}_{fold}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, True)
        print(f"f1 = {f1} acc = {acc}")
        acc_all.append(acc)
        f1_all.append(f1)
        print(acc, f1)
        pd.DataFrame(f1_all).to_csv("f1_data_arousal_psv_noleak_common.csv")
        pd.DataFrame(acc_all).to_csv("acc_data_arousal_psv_noleak_common.csv")

training...:   0%|          | 0/228 [00:00<?, ?it/s]

0.6206198590889311


validating...:   0%|          | 0/57 [00:00<?, ?it/s]

0.6280920010592256


predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

...

predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

0.6117645894203515 0.6756441885964912


predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

0.680921052631579
[[2342 2560]
 [2096 7594]]
              precision    recall  f1-score   support

           0       0.53      0.48      0.50      4902
           1       0.75      0.78      0.77      9690

    accuracy                           0.68     14592
   macro avg       0.64      0.63      0.63     14592
weighted avg       0.67      0.68      0.68     14592

f1 = 0.6334344072199991 acc = 0.680921052631579
0.680921052631579 0.6334344072199991


KeyboardInterrupt: ignored

In [ ]:
type_emotion = 1
acc_all_subs = []
f1_all_subs = []
for sub in range(32):
    acc_sub = []
    f1_sub = []
    for fold in range(0, 1):    
        # print(inds_train[sub:sub + 1, fold])
        # print(inds_test[sub : sub + 1, fold])
        # print(inds_train[sub : sub + 1, fold].sum())
        # print(inds_test[sub : sub + 1, fold].sum())
        # train_dataset = EmotionDataset((data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, interval, shift, inds_train[sub:sub+1, fold])
        # train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
        #                           pin_memory=True, shuffle=True, drop_last=True)


        val_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, interval, shift, inds_test[sub : sub + 1, fold])
        val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                                   pin_memory=True, shuffle=False, drop_last=False)
        
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        # model = get_model()
        # model.apply(initialize_weights)
        # criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
        # #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
        # optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
        description = "arousal_psv_noleak"
        # train_loop(type_emotion, description, fold, 5)пео

        
        model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"{description}_{fold}.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
        model.load_state_dict(model_state['model_state_dict'])
        
        acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, False)
        print(f"f1 = {f1} acc = {acc}")
        acc_sub.append(acc)
        f1_sub.append(f1)
        print(acc, f1)
    acc_all_subs.append(acc_sub)   
    f1_all_subs.append(f1_sub)   
    pd.DataFrame(f1_all_subs).to_csv("f1_data_arousal_psv_noleak_per_subs.csv")
    pd.DataFrame(acc_all_subs).to_csv("acc_data_arousal_psv_noleak_per_subs.csv")

predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.5787351977828168 acc = 0.6140350877192983
0.6140350877192983 0.5787351977828168


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.43379314358884813 acc = 0.43859649122807015
0.43859649122807015 0.43379314358884813


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

...

f1 = 0.42857142857142855 acc = 0.75
0.75 0.42857142857142855


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.4393246842038519 acc = 0.5109649122807017
0.5109649122807017 0.4393246842038519


predicting...:   0%|          | 0/2 [00:00<?, ?it/s]

f1 = 0.4382648733793772 acc = 0.7324561403508771
0.7324561403508771 0.4382648733793772


In [ ]:
print(np.mean(np.array(f1_all_subs).mean(axis = 1)))
print(np.mean(np.array(acc_all_subs).mean(axis = 1)))

0.4961225127712699
0.680921052631579


In [ ]:
# type_emotion = 0
# from sklearn.model_selection import StratifiedKFold
# interval = 3
# shift = 1
# k = 5
# inds_train = []
# inds_test = []
# for sub in range(32):  
#     X = np.arange(40)
#     y = np.array(labels_bin[sub][:, type_emotion])
#     skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
#     balanced_split = skf.split(X, y)
#     acc_sub = []
#     f1_sub = []
#     for ind_train, ind_test in  balanced_split:
#         #print(ind_train, ind_test)
#         #количестов меток 1 в трейне и тесте
#         #print(sum(labels_bin[sub][ind_train, type_emotion]), sum(labels_bin[sub][ind_test, type_emotion]))
#         #print((labels_bin[sub][ind_train, type_emotion]), (labels_bin[sub][ind_test, type_emotion]))
#         inds_train.append(ind_train)
#         inds_test.append(ind_test)

#         train_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[sub ], transforms, interval, shift, inds_train)
#         train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                                    pin_memory=True, shuffle=True, drop_last=True)


#         val_dataset = EmotionDataset(data, labels_bin, transforms, interval, shift, inds_test)
#         val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                                    pin_memory=True, shuffle=False, drop_last=False)
        
#         device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#         model = get_model()
#         model.apply(initialize_weights)
#         train_loop()

#         model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_psv_noleak.tgz"))
#         #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
#         model.load_state_dict(model_state['model_state_dict'])
        
#         acc, f1 = calculate_predictions(model, val_dataloader)
#         print(f"f1 = {f1} acc = {acc}")
#         acc_sub.append(acc)
#         f1_sub.append(f1)
#         print(acc, f1)
#     acc_all.append(acc_sub)     
#     f1_all.append(f1_sub)     
#     pd.DataFrame(f1_all).to_csv("f1_data_psv_noleak.csv")
#     pd.DataFrame(acc_all).to_csv("acc_data_psv_noleak.csv")

In [ ]:
# from sklearn.model_selection import StratifiedKFold 
# from sklearn.metrics import f1_score, accuracy_score


# #def get_mertics_per_subject():
# k  = 5
# type_emotion = 1

# acc_all = []
# f1_all = []
# for sub in range(0,1):
#     print(f"*******{sub}*********")

#     args.batch_size = 100
#     indexes = np.arange(NVIDEOS * 1 * LEN_RECORD_IN_SECONDS)
#     n = len(indexes)
#     #X = np.arange(40)
#     y = []
#     for nvideo in range(NVIDEOS):
#         y.extend(60 * [labels_bin[sub][nvideo, type_emotion]])
    

#     skf = StratifiedKFold(n_splits=k, random_state=None, shuffle=True)
#     balanced_split = skf.split(indexes, y)
#     acc_sub = []
#     f1_sub = []
#     for fold,  (inds_train, inds_test) in  enumerate(balanced_split):
#         print(f"fold = {fold}")
#         #print(inds_train, inds_test)
#         #print(sum(labels_bin[sub][inds_train, type_emotion]))
#         #print(sum(labels_bin[sub][inds_test, type_emotion]))
#         args.batch_size = 100
#         #transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
#         #transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
#         transforms = [to_head_matrix(),ToTensor()] 
#         #transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
#         #transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   

#         train_dataset = EmotionDataset(data[sub : sub + 1], labels_bin[sub : sub + 1], transforms, inds_train)
#         train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                                     pin_memory=True, shuffle=True, drop_last=True)

#         val_dataset = EmotionDataset(data[sub: sub + 1], labels_bin[sub  : sub + 1], transforms,  inds_test)
#         val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                               pin_memory=True, shuffle=False, drop_last=False)
#         device  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#         model = get_model(batch_norm = True, reg_extractor = True, ass_extractor = True)
#         model.apply(initialize_weights)
#         criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
#         #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
#         optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
#         train_loop(f'batchnorm_Yes_batch_100_Adam_lr=3e4_{sub}_{fold}_subject2', 1, 50)
#         description = f'batchnorm_Yes_batch_100_Adam_lr=3e4_{sub}_{fold}_subject2'
#         model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
#         #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
#         model.load_state_dict(model_state['model_state_dict'])
#         acc, f1 = calculate_predictions(model, val_dataloader, type_emotion)
#         print(f"f1 = {f1} acc = {acc}")
#         acc_sub.append(acc)
#         f1_sub.append(f1)
#         print(acc, f1)
#     acc_all.append(acc_sub)     
#     f1_all.append(f1_sub) 